# Imports

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import requests
from furl import furl
from math import ceil
import json
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.util as util
import sys
import pprint
import time

from sklearn.preprocessing import StandardScaler

from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_similarity


pp = pprint.PrettyPrinter()
%config InlineBackend.figure_format ='retina'

### Setting Global Variables:

In [2]:
SPOTIFY_CLIENT_ID='MY_CLIENT_ID'
SPOTIFY_CLIENT_SECRET='MY_CLIENT_SECRET'
SPOTIFY_REDIRECT_URI='MY_REDIRECT_URI'

In [3]:
df = pd.read_csv('../00_my-library-analysis/data/likedtracks_info_audiofeatures.csv').drop("Unnamed: 0", axis=1)

In [4]:
df['album_releasedate'] = pd.to_datetime(df['album_releasedate'], infer_datetime_format=True)

In [5]:
df['year'] = df['album_releasedate'].dt.year

In [6]:
df.shape

(3300, 33)

In [7]:
df = df.sort_values(by='album_releasedate', ascending=False).drop_duplicates(subset=['title', 'artist'], keep='first')

In [8]:
full_df = df.copy()

In [9]:
df.head(1)

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,full_time_added,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,year
91,1Z6zwgVuT2ytuP2GWszvZ6,Anaphylaxis,PUP,6A7uqgC2N1nUhrCLAytHxN,Anaphylaxis,5SH4FPOyMUYOgfNSjvD2Kf,2020-04-27,51,2020-04-27T22:33:23Z,2020-04-27 22:33:23,...,0.577,126.03,audio_features,1Z6zwgVuT2ytuP2GWszvZ6,spotify:track:1Z6zwgVuT2ytuP2GWszvZ6,https://api.spotify.com/v1/tracks/1Z6zwgVuT2yt...,https://api.spotify.com/v1/audio-analysis/1Z6z...,197829,3,2020


In [10]:
df = df[['track_id',
 'popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
'year']]

In [11]:
df.set_index('track_id', inplace=True)

In [12]:
df.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
track_id,,,,,,,,,,,,,,,
1Z6zwgVuT2ytuP2GWszvZ6,51,0.466,0.961,11,-2.917,1,0.0776,0.000009,0.002100,0.1440,0.577,126.030,197829,3,2020
2UrSdFSG1rTuzCwAkeP0ou,60,0.597,0.225,9,-11.588,0,0.0318,0.823000,0.000035,0.1420,0.119,88.956,183748,4,2020
4HfmEHdEJ52Y8XiL61h9SV,54,0.554,0.772,7,-3.939,1,0.0368,0.004820,0.000000,0.5360,0.413,112.045,246320,3,2020
66SAJfR6Pc8zqZEz9hglFT,30,0.459,0.644,9,-7.832,0,0.1180,0.176000,0.000016,0.1740,0.380,96.624,325669,4,2020
11tNXGkKXQvqQrNBKci4zK,16,0.517,0.963,7,-5.668,1,0.0634,0.022200,0.033400,0.0556,0.617,169.013,176847,4,2020


In [13]:
def get_track_title(row):
     return row['title'] + ' - ' + row['artist'] + ' - ' + row['track_id']
# full_FRIEND_info_audiofeatures['formatted_title'] = full_FRIEND_info_audiofeatures.apply(get_track_title)

In [14]:
full_df['formatted_title'] = full_df.apply(get_track_title, axis=1)


In [15]:
hannah_track_titles = dict(zip(full_df['track_id'], full_df['formatted_title']))

In [16]:
hannah_track_titles

{'1Z6zwgVuT2ytuP2GWszvZ6': 'Anaphylaxis - PUP - 1Z6zwgVuT2ytuP2GWszvZ6',
 '2UrSdFSG1rTuzCwAkeP0ou': 'Heart-Shaped Box - Glass Animals - 2UrSdFSG1rTuzCwAkeP0ou',
 '4HfmEHdEJ52Y8XiL61h9SV': 'everyone blooms - The Front Bottoms - 4HfmEHdEJ52Y8XiL61h9SV',
 '66SAJfR6Pc8zqZEz9hglFT': 'La Grippe 2020 - Squirrel Nut Zippers - 66SAJfR6Pc8zqZEz9hglFT',
 '11tNXGkKXQvqQrNBKci4zK': 'Porkupine - Sweet Crude - 11tNXGkKXQvqQrNBKci4zK',
 '4K1hUWsmBrAnSDE9ADDjLa': 'Diamond - Mapei - 4K1hUWsmBrAnSDE9ADDjLa',
 '7DLCgotDOZZyiW4cuL1sgn': 'Cop Car - Mitski - 7DLCgotDOZZyiW4cuL1sgn',
 '5Yj2d9bUigm12913o9IbTE': 'What We Have Is You - Sterling K. Brown - 5Yj2d9bUigm12913o9IbTE',
 '3vq0YrXFXMGqyqiKwD4dAD': 'Grrrl Like - Dope Saint Jude - 3vq0YrXFXMGqyqiKwD4dAD',
 '4uMcM69KR2ZveqWG2l8JSG': 'Wolf Chase - Daniel Rojas - 4uMcM69KR2ZveqWG2l8JSG',
 '1blEKwX5YDI8XK3u79qMxv': 'Dark Saturday - Dirt Road Version - Metric - 1blEKwX5YDI8XK3u79qMxv',
 '6iewIywVK4V0SQfSCZonCR': 'Resentment (feat. Sturgill Simpson, Brian Wilso

In [17]:
ss = StandardScaler()
df_scaled = ss.fit_transform(df)

In [18]:
sparse_df = sparse.csr_matrix(df_scaled)

In [19]:
recommender = cosine_similarity(sparse_df)

In [20]:
recommender_df = pd.DataFrame(recommender, columns=df.index, index=df.index)
recommender_df.head()

track_id,1Z6zwgVuT2ytuP2GWszvZ6,2UrSdFSG1rTuzCwAkeP0ou,4HfmEHdEJ52Y8XiL61h9SV,66SAJfR6Pc8zqZEz9hglFT,11tNXGkKXQvqQrNBKci4zK,4K1hUWsmBrAnSDE9ADDjLa,7DLCgotDOZZyiW4cuL1sgn,5Yj2d9bUigm12913o9IbTE,3vq0YrXFXMGqyqiKwD4dAD,4uMcM69KR2ZveqWG2l8JSG,...,7wL9EVloIobMw22rT7Ve8x,0kVJk1W8f8DOGOdqr11qOo,7hxZF4jETnE5Q75rKQnMjE,42yexCY4dCftowtAZXuAIj,11VwZwNF29HrqwalYUMitb,2fDHuS1PTkHBbCWWZF1ph9,0MKqZjDlfeltCfcrtIDj47,7s25THrKz86DM225dOYwnr,72T5smavl9ONfOwI0gHf1A,1oG2vWELiGjIqxwqGcyqwF
track_id,,,,,,,,,,,,,,,,,,,,,
1Z6zwgVuT2ytuP2GWszvZ6,1.000000,-0.086722,0.686584,0.274246,0.574704,0.332692,0.314056,0.235363,0.255390,-0.115747,...,-0.318348,-0.218234,-0.361471,-0.495002,-0.524553,-0.318471,-0.553664,-0.210104,-0.526805,-0.097710
2UrSdFSG1rTuzCwAkeP0ou,-0.086722,1.000000,-0.017481,0.413599,-0.370269,0.116756,0.525960,0.448626,0.373851,0.167074,...,-0.250639,-0.297795,0.011290,-0.249605,0.051764,-0.343073,-0.128808,-0.374852,-0.232849,-0.611893
4HfmEHdEJ52Y8XiL61h9SV,0.686584,-0.017481,1.000000,0.197670,0.140448,0.217958,0.216289,0.265507,0.085301,-0.065592,...,-0.294922,-0.292596,-0.380681,-0.484315,-0.405920,0.078326,-0.549526,-0.255192,-0.475473,-0.302997
66SAJfR6Pc8zqZEz9hglFT,0.274246,0.413599,0.197670,1.000000,0.077996,0.410025,0.641244,-0.233491,0.427256,-0.124832,...,-0.555622,-0.296312,-0.513366,-0.558609,-0.409787,-0.505579,-0.652587,-0.597230,-0.437868,-0.492920
11tNXGkKXQvqQrNBKci4zK,0.574704,-0.370269,0.140448,0.077996,1.000000,0.562890,0.152226,-0.050019,0.217698,-0.046589,...,-0.427054,-0.232729,-0.540552,-0.513772,-0.715633,-0.461547,-0.480113,-0.248199,-0.389988,-0.013934


In [21]:
def get_track_title(row):
     return row['title'] + ' - ' + row['artist']
# full_df['formatted_title'] = full_df.apply(get_track_title)

In [22]:
full_df['formatted_title'] = full_df.apply(get_track_title, axis=1)

In [23]:
track_titles = dict(zip(full_df['track_id'], full_df['formatted_title']))

In [24]:
recommender_df = recommender_df.rename(track_titles).rename(track_titles, axis=1)


In [25]:
recommender_df.head()

track_id,Anaphylaxis - PUP,Heart-Shaped Box - Glass Animals,everyone blooms - The Front Bottoms,La Grippe 2020 - Squirrel Nut Zippers,Porkupine - Sweet Crude,Diamond - Mapei,Cop Car - Mitski,What We Have Is You - Sterling K. Brown,Grrrl Like - Dope Saint Jude,Wolf Chase - Daniel Rojas,...,Man in Black - Johnny Cash,For What It's Worth - Sergio Mendes & Brasil '66,I Walk the Line - Johnny Cash,' Cello Song - Nick Drake,Pale Blue Eyes - The Velvet Underground,"Folsom Prison Blues - Live at Folsom State Prison, Folsom, CA - January 1968 - Johnny Cash","Ruby, Are You Mad At Your Man? - The New Lost City Ramblers",Respect - Aretha Franklin,You're Dead - Norma Tanega,You Really Got Me - The Kinks
track_id,,,,,,,,,,,,,,,,,,,,,
Anaphylaxis - PUP,1.000000,-0.086722,0.686584,0.274246,0.574704,0.332692,0.314056,0.235363,0.255390,-0.115747,...,-0.318348,-0.218234,-0.361471,-0.495002,-0.524553,-0.318471,-0.553664,-0.210104,-0.526805,-0.097710
Heart-Shaped Box - Glass Animals,-0.086722,1.000000,-0.017481,0.413599,-0.370269,0.116756,0.525960,0.448626,0.373851,0.167074,...,-0.250639,-0.297795,0.011290,-0.249605,0.051764,-0.343073,-0.128808,-0.374852,-0.232849,-0.611893
everyone blooms - The Front Bottoms,0.686584,-0.017481,1.000000,0.197670,0.140448,0.217958,0.216289,0.265507,0.085301,-0.065592,...,-0.294922,-0.292596,-0.380681,-0.484315,-0.405920,0.078326,-0.549526,-0.255192,-0.475473,-0.302997
La Grippe 2020 - Squirrel Nut Zippers,0.274246,0.413599,0.197670,1.000000,0.077996,0.410025,0.641244,-0.233491,0.427256,-0.124832,...,-0.555622,-0.296312,-0.513366,-0.558609,-0.409787,-0.505579,-0.652587,-0.597230,-0.437868,-0.492920
Porkupine - Sweet Crude,0.574704,-0.370269,0.140448,0.077996,1.000000,0.562890,0.152226,-0.050019,0.217698,-0.046589,...,-0.427054,-0.232729,-0.540552,-0.513772,-0.715633,-0.461547,-0.480113,-0.248199,-0.389988,-0.013934


In [26]:
full_df[full_df['formatted_title'] == 'Lemon Eyes - Meg Myers']

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,full_time_added,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,year,formatted_title
3,5sjEQjDkEYOr1RB6n5acXD,Lemon Eyes,Meg Myers,0W8xe7IqAPlnBRMUpWOUuJ,Sorry,14Q6NvjUSthc0Xgb5EgtQd,2015-09-18,48,2020-06-02T22:24:27Z,2020-06-02 22:24:27,...,139.937,audio_features,5sjEQjDkEYOr1RB6n5acXD,spotify:track:5sjEQjDkEYOr1RB6n5acXD,https://api.spotify.com/v1/tracks/5sjEQjDkEYOr...,https://api.spotify.com/v1/audio-analysis/5sjE...,219827,4,2015,Lemon Eyes - Meg Myers


In [27]:
pd.DataFrame(recommender_df['Lemon Eyes - Meg Myers'].sort_values(ascending=False)).head(20) 

,Lemon Eyes - Meg Myers
track_id,
Lemon Eyes - Meg Myers,1.000000
I Got the Juice (feat. Pharrell Williams) - Janelle Monáe,0.881473
bodyache - Purity Ring,0.867470
Water Fountain - Tune-Yards,0.859199
Like a Girl - Lizzo,0.859105
Otter Pop (feat. Hollis) - Shawn Wasabi,0.846320
Make Me Feel - Janelle Monáe,0.835487
Classy Girls - The Lumineers,0.817422
Trainwreck - BANKS,0.803052


In [28]:
full_df.head()

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,full_time_added,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,year,formatted_title
91,1Z6zwgVuT2ytuP2GWszvZ6,Anaphylaxis,PUP,6A7uqgC2N1nUhrCLAytHxN,Anaphylaxis,5SH4FPOyMUYOgfNSjvD2Kf,2020-04-27,51,2020-04-27T22:33:23Z,2020-04-27 22:33:23,...,126.030,audio_features,1Z6zwgVuT2ytuP2GWszvZ6,spotify:track:1Z6zwgVuT2ytuP2GWszvZ6,https://api.spotify.com/v1/tracks/1Z6zwgVuT2yt...,https://api.spotify.com/v1/audio-analysis/1Z6z...,197829,3,2020,Anaphylaxis - PUP
119,2UrSdFSG1rTuzCwAkeP0ou,Heart-Shaped Box,Glass Animals,4yvcSjfu4PC0CYQyLy4wSq,Quarantine Covers,1PoNxdvXr6C4XM4cr6Fr6u,2020-04-17,60,2020-04-23T02:45:40Z,2020-04-23 02:45:40,...,88.956,audio_features,2UrSdFSG1rTuzCwAkeP0ou,spotify:track:2UrSdFSG1rTuzCwAkeP0ou,https://api.spotify.com/v1/tracks/2UrSdFSG1rTu...,https://api.spotify.com/v1/audio-analysis/2UrS...,183748,4,2020,Heart-Shaped Box - Glass Animals
120,4HfmEHdEJ52Y8XiL61h9SV,everyone blooms,The Front Bottoms,5ictveRyhWRs8Gt8Dvt1hS,everyone blooms,26KAku0l1AFIgT65GehP34,2020-04-17,54,2020-04-23T02:41:20Z,2020-04-23 02:41:20,...,112.045,audio_features,4HfmEHdEJ52Y8XiL61h9SV,spotify:track:4HfmEHdEJ52Y8XiL61h9SV,https://api.spotify.com/v1/tracks/4HfmEHdEJ52Y...,https://api.spotify.com/v1/audio-analysis/4Hfm...,246320,3,2020,everyone blooms - The Front Bottoms
114,66SAJfR6Pc8zqZEz9hglFT,La Grippe 2020,Squirrel Nut Zippers,0LIll5i3kwo5A3IDpipgkS,La Grippe 2020,6dgpqDdpF6o1MYbMSZK2Wm,2020-04-03,30,2020-04-25T19:06:55Z,2020-04-25 19:06:55,...,96.624,audio_features,66SAJfR6Pc8zqZEz9hglFT,spotify:track:66SAJfR6Pc8zqZEz9hglFT,https://api.spotify.com/v1/tracks/66SAJfR6Pc8z...,https://api.spotify.com/v1/audio-analysis/66SA...,325669,4,2020,La Grippe 2020 - Squirrel Nut Zippers
118,11tNXGkKXQvqQrNBKci4zK,Porkupine,Sweet Crude,6SkFGEcHApSGM7xXGn5u64,Porkupine,36hQmkSJ9D52A1o0PrG6Cv,2020-04-03,16,2020-04-23T02:53:35Z,2020-04-23 02:53:35,...,169.013,audio_features,11tNXGkKXQvqQrNBKci4zK,spotify:track:11tNXGkKXQvqQrNBKci4zK,https://api.spotify.com/v1/tracks/11tNXGkKXQvq...,https://api.spotify.com/v1/audio-analysis/11tN...,176847,4,2020,Porkupine - Sweet Crude


In [29]:
recommender_df

track_id,Anaphylaxis - PUP,Heart-Shaped Box - Glass Animals,everyone blooms - The Front Bottoms,La Grippe 2020 - Squirrel Nut Zippers,Porkupine - Sweet Crude,Diamond - Mapei,Cop Car - Mitski,What We Have Is You - Sterling K. Brown,Grrrl Like - Dope Saint Jude,Wolf Chase - Daniel Rojas,...,Man in Black - Johnny Cash,For What It's Worth - Sergio Mendes & Brasil '66,I Walk the Line - Johnny Cash,' Cello Song - Nick Drake,Pale Blue Eyes - The Velvet Underground,"Folsom Prison Blues - Live at Folsom State Prison, Folsom, CA - January 1968 - Johnny Cash","Ruby, Are You Mad At Your Man? - The New Lost City Ramblers",Respect - Aretha Franklin,You're Dead - Norma Tanega,You Really Got Me - The Kinks
track_id,,,,,,,,,,,,,,,,,,,,,
Anaphylaxis - PUP,1.000000,-0.086722,0.686584,0.274246,0.574704,0.332692,0.314056,0.235363,0.255390,-0.115747,...,-0.318348,-0.218234,-0.361471,-0.495002,-0.524553,-0.318471,-0.553664,-0.210104,-0.526805,-0.097710
Heart-Shaped Box - Glass Animals,-0.086722,1.000000,-0.017481,0.413599,-0.370269,0.116756,0.525960,0.448626,0.373851,0.167074,...,-0.250639,-0.297795,0.011290,-0.249605,0.051764,-0.343073,-0.128808,-0.374852,-0.232849,-0.611893
everyone blooms - The Front Bottoms,0.686584,-0.017481,1.000000,0.197670,0.140448,0.217958,0.216289,0.265507,0.085301,-0.065592,...,-0.294922,-0.292596,-0.380681,-0.484315,-0.405920,0.078326,-0.549526,-0.255192,-0.475473,-0.302997
La Grippe 2020 - Squirrel Nut Zippers,0.274246,0.413599,0.197670,1.000000,0.077996,0.410025,0.641244,-0.233491,0.427256,-0.124832,...,-0.555622,-0.296312,-0.513366,-0.558609,-0.409787,-0.505579,-0.652587,-0.597230,-0.437868,-0.492920
Porkupine - Sweet Crude,0.574704,-0.370269,0.140448,0.077996,1.000000,0.562890,0.152226,-0.050019,0.217698,-0.046589,...,-0.427054,-0.232729,-0.540552,-0.513772,-0.715633,-0.461547,-0.480113,-0.248199,-0.389988,-0.013934
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Folsom Prison Blues - Live at Folsom State Prison, Folsom, CA - January 1968 - Johnny Cash",-0.318471,-0.343073,0.078326,-0.505579,-0.461547,-0.413805,-0.498560,-0.269190,-0.297483,-0.227912,...,0.801383,0.672707,0.638161,0.559719,0.594372,1.000000,0.565937,0.695238,0.717049,0.583016
"Ruby, Are You Mad At Your Man? - The New Lost City Ramblers",-0.553664,-0.128808,-0.549526,-0.652587,-0.480113,-0.568047,-0.510952,-0.098971,-0.346166,-0.116356,...,0.790281,0.689299,0.861828,0.770455,0.846354,0.565937,1.000000,0.664131,0.717346,0.645786
Respect - Aretha Franklin,-0.210104,-0.374852,-0.255192,-0.597230,-0.248199,-0.369155,-0.423527,-0.188306,-0.075657,-0.180333,...,0.892979,0.696498,0.795968,0.663267,0.667498,0.695238,0.664131,1.000000,0.867614,0.813873


In [30]:
# def get_big_match(song):
#     return recommender_df[(recommender_df[song] > .6) & (recommender_df[song] < 1)].sort_values(by=song, ascending=False)

In [31]:
def get_big_match(song):
    return recommender_df[(recommender_df[song] > .8) & (recommender_df[song] < 1)][[song]].sort_values(by=song, ascending=False)


In [32]:
get_big_match('Lemon Eyes - Meg Myers')

track_id,Lemon Eyes - Meg Myers
track_id,
I Got the Juice (feat. Pharrell Williams) - Janelle Monáe,0.881473
bodyache - Purity Ring,0.867470
Water Fountain - Tune-Yards,0.859199
Like a Girl - Lizzo,0.859105
Otter Pop (feat. Hollis) - Shawn Wasabi,0.846320
Make Me Feel - Janelle Monáe,0.835487
Classy Girls - The Lumineers,0.817422
Trainwreck - BANKS,0.803052


### Getting Tracks From FRIEND's Library:

In [127]:
username = 'MY_USERNAME'
scope = ''

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

FRIEND_tracks = {}

if token:
    sp = spotipy.Spotify(auth=token)
    for i in range(10):
        results  = sp.playlist_tracks('FRIEND_PLAYLIST', limit=100, offset=(i*100))
        for item in results['items']:
            track = item['track']
#            track_info = {track['name']: [track['artists'][0]['name'], track['album']['name']]}
            FRIEND_tracks[track['id']] = [track['name'], # track title
                                             track['artists'][0]['name'], # artist name
                                             track['artists'][0]['id'], # artist id
                                             track['album']['name'], # album title
                                             track['album']['id'], # album id
                                             track['album']['release_date'], # date released 
                                             track['popularity'],
                                             item['added_at']] # time added 
#             track_info = {track['name']: [track['artists'][0]['name'], track['album']['name']]}
#             all_saved_tracks[track['name']] = [track['artists'][0]['name'], track['album']['name']]
#            print(track['name'] + ' - ' + track['artists'][0]['name'])

In [128]:
FRIEND_tracks

{'54jUzTyKnfpouJMuEgcO1s': ['Teen Challenge',
  'Great Grandpa',
  '1Hs5RG6WIwUSJLxRYWaOW6',
  'Plastic Cough',
  '2VvgqYs7lEDT0wVOsYlZpR',
  '2017-07-07',
  36,
  '2020-06-08T21:36:17Z'],
 '6kpJGFFVsloSGd6PbEeve9': ['Pray for Rain',
  'The Ergs!',
  '0DWMLvlNTK198Lq3wUwkVN',
  'Dork Rock Cork Rod',
  '1kEXNfgChBAOabyB87tUU3',
  '2004-06-01',
  22,
  '2020-06-08T21:36:25Z'],
 '7gr3NOht9rVlnwF85QVNEs': ['Bite Your Tongue, Hard',
  'GILT',
  '59nCDPMkSmF4iyBkMwzpzx',
  'Shurwood / Gilt Split',
  '5EQ7AJ2YmGV3twlAt2fRo7',
  '2018-06-21',
  5,
  '2020-06-08T21:36:31Z'],
 '0yny9c2sLkQVKUmJiaFv17': ['you know who you are',
  'sandcastles.',
  '5Wg5VI0j7bRKPrTevdinxx',
  'die alone',
  '7vaL91fmKYYtzQ38J0clZ3',
  '2016-10-14',
  0,
  '2020-06-08T21:36:39Z'],
 '2zgaBa7mzGyww2lsIULIqY': ['Bipolar Planes',
  'This Is My Fist',
  '3iMu6Xdhv7cuIuRaUFrKKn',
  'A History of Rats',
  '4zhxy2SDBP1UeeD3GI2woa',
  '2006-08-06',
  3,
  '2020-06-08T21:37:05Z'],
 '0XioZu8ek6gr8TJLH2ZaRe': ["I'm Terrorfied!

In [129]:
FRIEND_info = pd.DataFrame.from_dict(FRIEND_tracks, orient='index', 
                                                  columns=['title', 
                                                           'artist', 
                                                           'artist_id', 
                                                           'album', 
                                                           'album_id', 
                                                           'album_releasedate',
                                                           'popularity',
                                                           'time_added']).reset_index()

In [130]:
FRIEND_info.rename({"index": "track_id"}, axis=1, inplace=True)

In [131]:
FRIEND_info.shape

(167, 9)

In [132]:
number_of_tracks = FRIEND_info.shape[0]

In [133]:
number_of_tracks

167

In [134]:
# List of tracks used to create audio features dataframe
FRIEND_list = FRIEND_info['track_id'].to_list()

In [135]:
username = 'MY_USERNAME'
scope = ''

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Usage: %s username" % (sys.argv[0],))
    sys.exit()

token = util.prompt_for_user_token(username, 
                                   scope,
                                  client_id=SPOTIFY_CLIENT_ID,
                                  client_secret=SPOTIFY_CLIENT_SECRET,
                                  redirect_uri=SPOTIFY_REDIRECT_URI)

likedtracks_features = []

if token:
    sp = spotipy.Spotify(auth=token)
    range_to_iterate = ceil(number_of_tracks/100)
    for i in range(range_to_iterate):
        begin = i*100
        end = i*100 + 99
        track_master_list = FRIEND_list[begin:end+1]
        list_features = sp.audio_features(track_master_list)
        likedtracks_features.extend(list_features)

In [136]:
FRIEND_features = pd.DataFrame(likedtracks_features)
FRIEND_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.337,0.813,9,-4.884,1,0.0578,0.04870,0.008840,0.110,0.4480,99.027,audio_features,54jUzTyKnfpouJMuEgcO1s,spotify:track:54jUzTyKnfpouJMuEgcO1s,https://api.spotify.com/v1/tracks/54jUzTyKnfpo...,https://api.spotify.com/v1/audio-analysis/54jU...,228115,4
1,0.214,0.995,4,-4.505,1,0.4140,0.02240,0.000000,0.328,0.1700,165.436,audio_features,6kpJGFFVsloSGd6PbEeve9,spotify:track:6kpJGFFVsloSGd6PbEeve9,https://api.spotify.com/v1/tracks/6kpJGFFVsloS...,https://api.spotify.com/v1/audio-analysis/6kpJ...,141653,4
2,0.559,0.233,0,-9.303,1,0.0403,0.16500,0.000173,0.219,0.2180,148.291,audio_features,7gr3NOht9rVlnwF85QVNEs,spotify:track:7gr3NOht9rVlnwF85QVNEs,https://api.spotify.com/v1/tracks/7gr3NOht9rVl...,https://api.spotify.com/v1/audio-analysis/7gr3...,97500,4
3,0.753,0.630,11,-5.637,0,0.0414,0.21200,0.000015,0.110,0.0534,130.129,audio_features,0yny9c2sLkQVKUmJiaFv17,spotify:track:0yny9c2sLkQVKUmJiaFv17,https://api.spotify.com/v1/tracks/0yny9c2sLkQV...,https://api.spotify.com/v1/audio-analysis/0yny...,236781,4
4,0.371,0.907,2,-4.563,1,0.0823,0.00159,0.000000,0.141,0.6810,113.019,audio_features,2zgaBa7mzGyww2lsIULIqY,spotify:track:2zgaBa7mzGyww2lsIULIqY,https://api.spotify.com/v1/tracks/2zgaBa7mzGyw...,https://api.spotify.com/v1/audio-analysis/2zga...,105213,4


In [137]:
FRIEND_features.shape

(167, 18)

#### Saving to CSV:

In [138]:
FRIEND_features.to_csv('data/FRIEND_features.csv')

### Combining Track Info with Audio Features:

In [139]:
FRIEND_info_audiofeatures = FRIEND_info.merge(FRIEND_features, left_on='track_id', right_on='id')

In [140]:
FRIEND_info_audiofeatures.to_csv('data/FRIEND_info_audiofeatures.csv')

In [141]:
FRIEND_info_audiofeatures

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,danceability,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,54jUzTyKnfpouJMuEgcO1s,Teen Challenge,Great Grandpa,1Hs5RG6WIwUSJLxRYWaOW6,Plastic Cough,2VvgqYs7lEDT0wVOsYlZpR,2017-07-07,36,2020-06-08T21:36:17Z,0.337,...,0.1100,0.4480,99.027,audio_features,54jUzTyKnfpouJMuEgcO1s,spotify:track:54jUzTyKnfpouJMuEgcO1s,https://api.spotify.com/v1/tracks/54jUzTyKnfpo...,https://api.spotify.com/v1/audio-analysis/54jU...,228115,4
1,6kpJGFFVsloSGd6PbEeve9,Pray for Rain,The Ergs!,0DWMLvlNTK198Lq3wUwkVN,Dork Rock Cork Rod,1kEXNfgChBAOabyB87tUU3,2004-06-01,22,2020-06-08T21:36:25Z,0.214,...,0.3280,0.1700,165.436,audio_features,6kpJGFFVsloSGd6PbEeve9,spotify:track:6kpJGFFVsloSGd6PbEeve9,https://api.spotify.com/v1/tracks/6kpJGFFVsloS...,https://api.spotify.com/v1/audio-analysis/6kpJ...,141653,4
2,7gr3NOht9rVlnwF85QVNEs,"Bite Your Tongue, Hard",GILT,59nCDPMkSmF4iyBkMwzpzx,Shurwood / Gilt Split,5EQ7AJ2YmGV3twlAt2fRo7,2018-06-21,5,2020-06-08T21:36:31Z,0.559,...,0.2190,0.2180,148.291,audio_features,7gr3NOht9rVlnwF85QVNEs,spotify:track:7gr3NOht9rVlnwF85QVNEs,https://api.spotify.com/v1/tracks/7gr3NOht9rVl...,https://api.spotify.com/v1/audio-analysis/7gr3...,97500,4
3,0yny9c2sLkQVKUmJiaFv17,you know who you are,sandcastles.,5Wg5VI0j7bRKPrTevdinxx,die alone,7vaL91fmKYYtzQ38J0clZ3,2016-10-14,0,2020-06-08T21:36:39Z,0.753,...,0.1100,0.0534,130.129,audio_features,0yny9c2sLkQVKUmJiaFv17,spotify:track:0yny9c2sLkQVKUmJiaFv17,https://api.spotify.com/v1/tracks/0yny9c2sLkQV...,https://api.spotify.com/v1/audio-analysis/0yny...,236781,4
4,2zgaBa7mzGyww2lsIULIqY,Bipolar Planes,This Is My Fist,3iMu6Xdhv7cuIuRaUFrKKn,A History of Rats,4zhxy2SDBP1UeeD3GI2woa,2006-08-06,3,2020-06-08T21:37:05Z,0.371,...,0.1410,0.6810,113.019,audio_features,2zgaBa7mzGyww2lsIULIqY,spotify:track:2zgaBa7mzGyww2lsIULIqY,https://api.spotify.com/v1/tracks/2zgaBa7mzGyw...,https://api.spotify.com/v1/audio-analysis/2zga...,105213,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162,17bmcZANoh4dfsbgq1HAZw,Plenty of Time,RVIVR,1nfqdFH2ssg8oV0TYG2h6p,The Joester Sessions,72ueGzZTyYhEeNqMh1GsLI,2011,10,2020-06-08T22:42:20Z,0.278,...,0.0859,0.4580,148.545,audio_features,17bmcZANoh4dfsbgq1HAZw,spotify:track:17bmcZANoh4dfsbgq1HAZw,https://api.spotify.com/v1/tracks/17bmcZANoh4d...,https://api.spotify.com/v1/audio-analysis/17bm...,169240,4
163,0Pk2xEYTdYKVUqoWKwMiN5,Can't stand It,RVIVR,1nfqdFH2ssg8oV0TYG2h6p,The Joester Sessions,72ueGzZTyYhEeNqMh1GsLI,2011,9,2020-06-08T22:42:20Z,0.204,...,0.4120,0.2620,174.360,audio_features,0Pk2xEYTdYKVUqoWKwMiN5,spotify:track:0Pk2xEYTdYKVUqoWKwMiN5,https://api.spotify.com/v1/tracks/0Pk2xEYTdYKV...,https://api.spotify.com/v1/audio-analysis/0Pk2...,176747,4
164,6iUjRWLpMIjzzEWAwVbX1s,Scrooged,RVIVR,1nfqdFH2ssg8oV0TYG2h6p,The Joester Sessions,72ueGzZTyYhEeNqMh1GsLI,2011,10,2020-06-08T22:42:20Z,0.190,...,0.1080,0.2240,165.918,audio_features,6iUjRWLpMIjzzEWAwVbX1s,spotify:track:6iUjRWLpMIjzzEWAwVbX1s,https://api.spotify.com/v1/tracks/6iUjRWLpMIjz...,https://api.spotify.com/v1/audio-analysis/6iUj...,220160,4
165,3XDfD2XjHP6bermfoAsfNa,Derailer,RVIVR,1nfqdFH2ssg8oV0TYG2h6p,The Joester Sessions,72ueGzZTyYhEeNqMh1GsLI,2011,9,2020-06-08T22:42:20Z,0.238,...,0.1120,0.5260,150.920,audio_features,3XDfD2XjHP6bermfoAsfNa,spotify:track:3XDfD2XjHP6bermfoAsfNa,https://api.spotify.com/v1/tracks/3XDfD2XjHP6b...,https://api.spotify.com/v1/audio-analysis/3XDf...,219787,4


# FRIEND Recommender?

In [142]:
FRIEND_info_audiofeatures['album_releasedate'] = pd.to_datetime(FRIEND_info_audiofeatures['album_releasedate'], infer_datetime_format=True)

In [143]:
FRIEND_info_audiofeatures['year'] = FRIEND_info_audiofeatures['album_releasedate'].dt.year

In [144]:
FRIEND_info_audiofeatures.shape

(167, 28)

In [145]:
FRIEND_info_audiofeatures = FRIEND_info_audiofeatures.sort_values(by='album_releasedate', ascending=False).drop_duplicates(subset=['title', 'artist'], keep='first')

In [146]:
full_FRIEND_info_audiofeatures = FRIEND_info_audiofeatures.copy()

In [147]:
FRIEND_info_audiofeatures.head(1)

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,danceability,...,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,year
120,4YvvrIJy1yuLrH46guye4m,Ex Cop,Bad Idols,7LMdSCdHzlUGwPlxhHS0OI,Can't Imagine,1UQE9p2LFvsaa3KzF5UH32,2019-09-20,0,2020-06-08T22:40:34Z,0.29,...,0.0479,85.005,audio_features,4YvvrIJy1yuLrH46guye4m,spotify:track:4YvvrIJy1yuLrH46guye4m,https://api.spotify.com/v1/tracks/4YvvrIJy1yuL...,https://api.spotify.com/v1/audio-analysis/4Yvv...,25080,5,2019


In [148]:
FRIEND_info_audiofeatures = FRIEND_info_audiofeatures[['track_id',
 'popularity',
 'danceability',
 'energy',
 'key',
 'loudness',
 'mode',
 'speechiness',
 'acousticness',
 'instrumentalness',
 'liveness',
 'valence',
 'tempo',
 'duration_ms',
 'time_signature',
'year']]

In [149]:
FRIEND_info_audiofeatures.set_index('track_id', inplace=True)

In [150]:
FRIEND_info_audiofeatures.head()

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
track_id,,,,,,,,,,,,,,,
4YvvrIJy1yuLrH46guye4m,0,0.290,0.818,9,-7.284,1,0.0348,0.002050,0.419,0.358,0.0479,85.005,25080,5,2019
2yAjekT6CrC6ZNgRqNvzuZ,0,0.402,0.906,4,-4.609,1,0.0734,0.004090,0.000,0.267,0.3550,163.908,96773,4,2019
3t0P1KfxMrJBqSe4kCm8hU,1,0.496,0.894,6,-4.153,1,0.0572,0.000042,0.000,0.313,0.4630,109.927,84480,4,2019
2qksUig2viYaDNdacQJJYt,0,0.494,0.945,2,-3.984,1,0.0477,0.000045,0.000,0.311,0.5700,96.911,98107,4,2019
0z7UWJes1jIUZHKo6LeEoI,0,0.332,0.953,4,-5.350,1,0.1100,0.000015,0.000,0.396,0.2340,120.541,53800,4,2019


In [161]:
FRIEND_info_audiofeatures[FRIEND_info_audiofeatures.index == '2rFB2r05YeRpsWAwIyUvWr']

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
track_id,,,,,,,,,,,,,,,
2rFB2r05YeRpsWAwIyUvWr,14,0.543,0.035,2,-17.445,1,0.037,0.922,0.000854,0.101,0.111,92.721,166848,4,2019


In [162]:
df[df.index == '2rFB2r05YeRpsWAwIyUvWr']

,popularity,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,year
track_id,,,,,,,,,,,,,,,
2rFB2r05YeRpsWAwIyUvWr,14,0.543,0.035,2,-17.445,1,0.037,0.922,0.000854,0.101,0.111,92.721,166848,4,2019


### Scaling Features

In [58]:
ss = StandardScaler()
FRIEND_info_audiofeatures_scaled = ss.fit_transform(FRIEND_info_audiofeatures)

In [59]:
sparse_FRIEND_info_audiofeatures = sparse.csr_matrix(FRIEND_info_audiofeatures_scaled)

## Creating Recommender for FRIEND

In [60]:
recommender = cosine_similarity(sparse_df, sparse_FRIEND_info_audiofeatures)

In [61]:
recommender_FRIEND_info_audiofeatures = pd.DataFrame(recommender, columns=FRIEND_info_audiofeatures.index, index=df.index)
recommender_FRIEND_info_audiofeatures



track_id,4YvvrIJy1yuLrH46guye4m,2yAjekT6CrC6ZNgRqNvzuZ,3t0P1KfxMrJBqSe4kCm8hU,2qksUig2viYaDNdacQJJYt,0z7UWJes1jIUZHKo6LeEoI,3dijvgdt9Z7WnxvIHUAUbF,6bbPSlmwFf5pOvuuq9z6uf,0wwmeNYycR31R8HmdP3a0b,1MDF2WWoTVxDLtcz6sgsh3,0reenPmynLrPz5pRWgrFsa,...,7fGVQN2dJNNUIv27Hqik3X,4RErgpX9FC3EQhWwZaBvbr,4TOoUhXQNUQ06yQRbbLTkA,3npWxtlCxtDlEttt4ffYgY,4ouQXSQAwQKB5GqpH06EHz,2FQULpNJjQsUfjmd6KUXAC,4IhBMMgZkrQybTajYXtmVE,1AOZ7vnaMCy9yx59xiE04f,2zgaBa7mzGyww2lsIULIqY,6kpJGFFVsloSGd6PbEeve9
track_id,,,,,,,,,,,,,,,,,,,,,
1Z6zwgVuT2ytuP2GWszvZ6,-0.127177,0.241317,0.337639,0.199581,0.192094,0.404738,-0.144708,0.431424,0.243423,0.452039,...,-0.010491,0.209178,-0.099958,-0.020897,-0.162664,-0.065485,-0.045035,-0.151699,-0.099007,-0.047208
2UrSdFSG1rTuzCwAkeP0ou,0.180458,-0.320432,-0.122340,-0.253966,-0.145095,-0.220006,0.077092,-0.053342,-0.095155,-0.052203,...,-0.463651,-0.277178,0.237784,-0.251922,-0.237399,-0.684128,-0.297004,-0.382363,-0.653893,-0.481938
4HfmEHdEJ52Y8XiL61h9SV,-0.126123,0.106351,0.261049,0.211176,0.222856,-0.017177,-0.009856,0.146478,0.154784,0.219073,...,0.290108,0.064536,-0.244364,0.215735,0.102890,0.130882,0.343067,0.029613,-0.310715,-0.141437
66SAJfR6Pc8zqZEz9hglFT,0.063517,-0.151093,0.023102,-0.046684,-0.048745,-0.028126,0.204036,0.142638,0.033903,0.199967,...,-0.275400,-0.234709,-0.104901,-0.116723,0.193580,-0.449136,-0.253983,-0.352055,-0.504294,-0.314743
11tNXGkKXQvqQrNBKci4zK,0.127943,0.776926,0.576893,0.466681,0.457108,0.839228,0.078888,0.591886,0.487699,0.622799,...,-0.186915,0.018925,-0.198036,-0.338049,-0.324097,0.082053,-0.270302,-0.243002,0.158282,-0.021625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2fDHuS1PTkHBbCWWZF1ph9,-0.250691,-0.382985,-0.282470,-0.156383,-0.245664,-0.515641,-0.492741,-0.356700,-0.417298,-0.444301,...,0.820821,0.401324,0.106083,0.236218,0.557616,0.628736,0.669932,0.478939,0.373176,0.360678
0MKqZjDlfeltCfcrtIDj47,-0.131610,-0.344951,-0.428761,-0.412874,-0.306176,-0.262774,-0.250288,-0.380408,-0.364201,-0.441064,...,0.308375,0.444474,0.281451,0.184913,0.140109,0.260758,0.105218,0.416887,0.392636,0.420983
7s25THrKz86DM225dOYwnr,-0.392766,-0.331197,-0.226510,-0.082447,-0.410349,-0.358632,-0.439384,-0.243855,-0.319652,-0.326836,...,0.513119,0.490626,0.342866,0.238223,0.333391,0.402167,0.480499,0.451697,0.680056,0.360187


Overall, our songs are **slightly** more similar than different:

In [62]:
recommender_FRIEND_info_audiofeatures.mean().mean()

0.003284804810638707

In [63]:
def get_track_title(row):
     return row['title'] + ' - ' + row['artist'] + ' - ' + row['track_id']
# full_FRIEND_info_audiofeatures['formatted_title'] = full_FRIEND_info_audiofeatures.apply(get_track_title)

In [64]:
full_FRIEND_info_audiofeatures['formatted_title'] = full_FRIEND_info_audiofeatures.apply(get_track_title, axis=1)


In [65]:
full_FRIEND_info_audiofeatures

,track_id,title,artist,artist_id,album,album_id,album_releasedate,popularity,time_added,danceability,...,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature,year,formatted_title
120,4YvvrIJy1yuLrH46guye4m,Ex Cop,Bad Idols,7LMdSCdHzlUGwPlxhHS0OI,Can't Imagine,1UQE9p2LFvsaa3KzF5UH32,2019-09-20,0,2020-06-08T22:40:34Z,0.290,...,85.005,audio_features,4YvvrIJy1yuLrH46guye4m,spotify:track:4YvvrIJy1yuLrH46guye4m,https://api.spotify.com/v1/tracks/4YvvrIJy1yuL...,https://api.spotify.com/v1/audio-analysis/4Yvv...,25080,5,2019,Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m
123,2yAjekT6CrC6ZNgRqNvzuZ,Freedom to Waste,Bad Idols,7LMdSCdHzlUGwPlxhHS0OI,Can't Imagine,1UQE9p2LFvsaa3KzF5UH32,2019-09-20,0,2020-06-08T22:40:34Z,0.402,...,163.908,audio_features,2yAjekT6CrC6ZNgRqNvzuZ,spotify:track:2yAjekT6CrC6ZNgRqNvzuZ,https://api.spotify.com/v1/tracks/2yAjekT6CrC6...,https://api.spotify.com/v1/audio-analysis/2yAj...,96773,4,2019,Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNg...
121,3t0P1KfxMrJBqSe4kCm8hU,Guilt,Bad Idols,7LMdSCdHzlUGwPlxhHS0OI,Can't Imagine,1UQE9p2LFvsaa3KzF5UH32,2019-09-20,1,2020-06-08T22:40:34Z,0.496,...,109.927,audio_features,3t0P1KfxMrJBqSe4kCm8hU,spotify:track:3t0P1KfxMrJBqSe4kCm8hU,https://api.spotify.com/v1/tracks/3t0P1KfxMrJB...,https://api.spotify.com/v1/audio-analysis/3t0P...,84480,4,2019,Guilt - Bad Idols - 3t0P1KfxMrJBqSe4kCm8hU
126,2qksUig2viYaDNdacQJJYt,I've Said Nothing,Bad Idols,7LMdSCdHzlUGwPlxhHS0OI,Can't Imagine,1UQE9p2LFvsaa3KzF5UH32,2019-09-20,0,2020-06-08T22:40:34Z,0.494,...,96.911,audio_features,2qksUig2viYaDNdacQJJYt,spotify:track:2qksUig2viYaDNdacQJJYt,https://api.spotify.com/v1/tracks/2qksUig2viYa...,https://api.spotify.com/v1/audio-analysis/2qks...,98107,4,2019,I've Said Nothing - Bad Idols - 2qksUig2viYaDN...
125,0z7UWJes1jIUZHKo6LeEoI,Dolly Parton Is My Mom,Bad Idols,7LMdSCdHzlUGwPlxhHS0OI,Can't Imagine,1UQE9p2LFvsaa3KzF5UH32,2019-09-20,0,2020-06-08T22:40:34Z,0.332,...,120.541,audio_features,0z7UWJes1jIUZHKo6LeEoI,spotify:track:0z7UWJes1jIUZHKo6LeEoI,https://api.spotify.com/v1/tracks/0z7UWJes1jIU...,https://api.spotify.com/v1/audio-analysis/0z7U...,53800,4,2019,Dolly Parton Is My Mom - Bad Idols - 0z7UWJes1...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2FQULpNJjQsUfjmd6KUXAC,Pizza Claus Is Comin' to Town,Bomb the Music Industry!,7mmU5GuOoyxoBAgOZkSVj7,Get Warmer,2kGge9kVEDKWLMb4YsrmIs,2007-09-11,20,2020-06-08T22:38:26Z,0.223,...,177.677,audio_features,2FQULpNJjQsUfjmd6KUXAC,spotify:track:2FQULpNJjQsUfjmd6KUXAC,https://api.spotify.com/v1/tracks/2FQULpNJjQsU...,https://api.spotify.com/v1/audio-analysis/2FQU...,191587,4,2007,Pizza Claus Is Comin' to Town - Bomb the Music...
88,4IhBMMgZkrQybTajYXtmVE,I Don't Love You Anymore,Bomb the Music Industry!,7mmU5GuOoyxoBAgOZkSVj7,Get Warmer,2kGge9kVEDKWLMb4YsrmIs,2007-09-11,46,2020-06-08T22:38:26Z,0.384,...,136.764,audio_features,4IhBMMgZkrQybTajYXtmVE,spotify:track:4IhBMMgZkrQybTajYXtmVE,https://api.spotify.com/v1/tracks/4IhBMMgZkrQy...,https://api.spotify.com/v1/audio-analysis/4IhB...,282093,4,2007,I Don't Love You Anymore - Bomb the Music Indu...
83,1AOZ7vnaMCy9yx59xiE04f,Bike Test 1 2 3,Bomb the Music Industry!,7mmU5GuOoyxoBAgOZkSVj7,Get Warmer,2kGge9kVEDKWLMb4YsrmIs,2007-09-11,22,2020-06-08T22:38:26Z,0.385,...,120.165,audio_features,1AOZ7vnaMCy9yx59xiE04f,spotify:track:1AOZ7vnaMCy9yx59xiE04f,https://api.spotify.com/v1/tracks/1AOZ7vnaMCy9...,https://api.spotify.com/v1/audio-analysis/1AOZ...,157560,4,2007,Bike Test 1 2 3 - Bomb the Music Industry! - 1...
4,2zgaBa7mzGyww2lsIULIqY,Bipolar Planes,This Is My Fist,3iMu6Xdhv7cuIuRaUFrKKn,A History of Rats,4zhxy2SDBP1UeeD3GI2woa,2006-08-06,3,2020-06-08T21:37:05Z,0.371,...,113.019,audio_features,2zgaBa7mzGyww2lsIULIqY,spotify:track:2zgaBa7mzGyww2lsIULIqY,https://api.spotify.com/v1/tracks/2zgaBa7mzGyw...,https://api.spotify.com/v1/audio-analysis/2zga...,105213,4,2006,Bipolar Planes - This Is My Fist - 2zgaBa7mzGy...


In [66]:
FRIEND_track_titles = dict(zip(full_FRIEND_info_audiofeatures['track_id'], full_FRIEND_info_audiofeatures['formatted_title']))

In [67]:
FRIEND_track_titles

{'4YvvrIJy1yuLrH46guye4m': 'Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m',
 '2yAjekT6CrC6ZNgRqNvzuZ': 'Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNgRqNvzuZ',
 '3t0P1KfxMrJBqSe4kCm8hU': 'Guilt - Bad Idols - 3t0P1KfxMrJBqSe4kCm8hU',
 '2qksUig2viYaDNdacQJJYt': "I've Said Nothing - Bad Idols - 2qksUig2viYaDNdacQJJYt",
 '0z7UWJes1jIUZHKo6LeEoI': 'Dolly Parton Is My Mom - Bad Idols - 0z7UWJes1jIUZHKo6LeEoI',
 '3dijvgdt9Z7WnxvIHUAUbF': 'Bile Bittersweet - Bad Idols - 3dijvgdt9Z7WnxvIHUAUbF',
 '6bbPSlmwFf5pOvuuq9z6uf': 'Like a Joke - Bad Idols - 6bbPSlmwFf5pOvuuq9z6uf',
 '0wwmeNYycR31R8HmdP3a0b': 'Saddest Girl in Town - Bad Idols - 0wwmeNYycR31R8HmdP3a0b',
 '1MDF2WWoTVxDLtcz6sgsh3': 'Nollie Big Spin - Bad Idols - 1MDF2WWoTVxDLtcz6sgsh3',
 '0reenPmynLrPz5pRWgrFsa': 'Breakdown Days - Bad Idols - 0reenPmynLrPz5pRWgrFsa',
 '4mpHviiBvLRglnRuomAJKL': 'Sticks and Stones - Bad Idols - 4mpHviiBvLRglnRuomAJKL',
 '6UlzxveYqDo4CdTTA88bSC': 'Gummy Worms - Bad Idols - 6UlzxveYqDo4CdTTA88bSC',
 '0qRjUWPHDfu1

In [68]:
FRIEND_rec = recommender_FRIEND_info_audiofeatures.rename(hannah_track_titles).rename(FRIEND_track_titles, axis=1)


In [69]:
# recommender_FRIEND_transposed = recommender_FRIEND_info_audiofeatures
# recommender_FRIEND_transposed

In [70]:
FRIEND_rec

track_id,Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m,Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNgRqNvzuZ,Guilt - Bad Idols - 3t0P1KfxMrJBqSe4kCm8hU,I've Said Nothing - Bad Idols - 2qksUig2viYaDNdacQJJYt,Dolly Parton Is My Mom - Bad Idols - 0z7UWJes1jIUZHKo6LeEoI,Bile Bittersweet - Bad Idols - 3dijvgdt9Z7WnxvIHUAUbF,Like a Joke - Bad Idols - 6bbPSlmwFf5pOvuuq9z6uf,Saddest Girl in Town - Bad Idols - 0wwmeNYycR31R8HmdP3a0b,Nollie Big Spin - Bad Idols - 1MDF2WWoTVxDLtcz6sgsh3,Breakdown Days - Bad Idols - 0reenPmynLrPz5pRWgrFsa,...,No Rest for the Whiny - Bomb the Music Industry! - 7fGVQN2dJNNUIv27Hqik3X,25 Hour Goddamn Telethon - Bomb the Music Industry! - 4RErgpX9FC3EQhWwZaBvbr,The Last Party (Foul) - Bomb the Music Industry! - 4TOoUhXQNUQ06yQRbbLTkA,Get Warmer - Bomb the Music Industry! - 3npWxtlCxtDlEttt4ffYgY,Never Trust a Man Without a Horribly Embarassing Secret - Bomb the Music Industry! - 4ouQXSQAwQKB5GqpH06EHz,Pizza Claus Is Comin' to Town - Bomb the Music Industry! - 2FQULpNJjQsUfjmd6KUXAC,I Don't Love You Anymore - Bomb the Music Industry! - 4IhBMMgZkrQybTajYXtmVE,Bike Test 1 2 3 - Bomb the Music Industry! - 1AOZ7vnaMCy9yx59xiE04f,Bipolar Planes - This Is My Fist - 2zgaBa7mzGyww2lsIULIqY,Pray for Rain - The Ergs! - 6kpJGFFVsloSGd6PbEeve9
track_id,,,,,,,,,,,,,,,,,,,,,
Anaphylaxis - PUP - 1Z6zwgVuT2ytuP2GWszvZ6,-0.127177,0.241317,0.337639,0.199581,0.192094,0.404738,-0.144708,0.431424,0.243423,0.452039,...,-0.010491,0.209178,-0.099958,-0.020897,-0.162664,-0.065485,-0.045035,-0.151699,-0.099007,-0.047208
Heart-Shaped Box - Glass Animals - 2UrSdFSG1rTuzCwAkeP0ou,0.180458,-0.320432,-0.122340,-0.253966,-0.145095,-0.220006,0.077092,-0.053342,-0.095155,-0.052203,...,-0.463651,-0.277178,0.237784,-0.251922,-0.237399,-0.684128,-0.297004,-0.382363,-0.653893,-0.481938
everyone blooms - The Front Bottoms - 4HfmEHdEJ52Y8XiL61h9SV,-0.126123,0.106351,0.261049,0.211176,0.222856,-0.017177,-0.009856,0.146478,0.154784,0.219073,...,0.290108,0.064536,-0.244364,0.215735,0.102890,0.130882,0.343067,0.029613,-0.310715,-0.141437
La Grippe 2020 - Squirrel Nut Zippers - 66SAJfR6Pc8zqZEz9hglFT,0.063517,-0.151093,0.023102,-0.046684,-0.048745,-0.028126,0.204036,0.142638,0.033903,0.199967,...,-0.275400,-0.234709,-0.104901,-0.116723,0.193580,-0.449136,-0.253983,-0.352055,-0.504294,-0.314743
Porkupine - Sweet Crude - 11tNXGkKXQvqQrNBKci4zK,0.127943,0.776926,0.576893,0.466681,0.457108,0.839228,0.078888,0.591886,0.487699,0.622799,...,-0.186915,0.018925,-0.198036,-0.338049,-0.324097,0.082053,-0.270302,-0.243002,0.158282,-0.021625
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"Folsom Prison Blues - Live at Folsom State Prison, Folsom, CA - January 1968 - Johnny Cash - 2fDHuS1PTkHBbCWWZF1ph9",-0.250691,-0.382985,-0.282470,-0.156383,-0.245664,-0.515641,-0.492741,-0.356700,-0.417298,-0.444301,...,0.820821,0.401324,0.106083,0.236218,0.557616,0.628736,0.669932,0.478939,0.373176,0.360678
"Ruby, Are You Mad At Your Man? - The New Lost City Ramblers - 0MKqZjDlfeltCfcrtIDj47",-0.131610,-0.344951,-0.428761,-0.412874,-0.306176,-0.262774,-0.250288,-0.380408,-0.364201,-0.441064,...,0.308375,0.444474,0.281451,0.184913,0.140109,0.260758,0.105218,0.416887,0.392636,0.420983
Respect - Aretha Franklin - 7s25THrKz86DM225dOYwnr,-0.392766,-0.331197,-0.226510,-0.082447,-0.410349,-0.358632,-0.439384,-0.243855,-0.319652,-0.326836,...,0.513119,0.490626,0.342866,0.238223,0.333391,0.402167,0.480499,0.451697,0.680056,0.360187


In [71]:
# pd.DataFrame(FRIEND_rec.mean(axis=1).sort_values(ascending=True)).head(20)

In [78]:
def get_big_match2(song):
    return FRIEND_rec[(FRIEND_rec[song] > .9) & (FRIEND_rec[song] < 1)][[song]].sort_values(by=song, ascending=False)


In [79]:
FRIEND_rec.shape

(3105, 167)

In [80]:
range(number_of_tracks)

range(0, 167)

In [81]:
# FRIEND_rec[0]

In [82]:
for i in range(167):
    song_to_compare = FRIEND_rec.columns[i]
    print('(' + song_to_compare + ')')
    print(len(get_big_match2(song_to_compare)))
    display(pd.DataFrame(get_big_match2(song_to_compare)))
    print()

(Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m)
0


track_id,Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m
track_id,



(Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNgRqNvzuZ)
0


track_id,Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNgRqNvzuZ
track_id,



(Guilt - Bad Idols - 3t0P1KfxMrJBqSe4kCm8hU)
0


...

track_id,Bike Test 1 2 3 - Bomb the Music Industry! - 1AOZ7vnaMCy9yx59xiE04f
track_id,



(Bipolar Planes - This Is My Fist - 2zgaBa7mzGyww2lsIULIqY)
0


track_id,Bipolar Planes - This Is My Fist - 2zgaBa7mzGyww2lsIULIqY
track_id,



(Pray for Rain - The Ergs! - 6kpJGFFVsloSGd6PbEeve9)
0


track_id,Pray for Rain - The Ergs! - 6kpJGFFVsloSGd6PbEeve9
track_id,


In [83]:
FRIEND_songs_for_playlist = {}

for i in range(number_of_tracks):
    song_to_compare = FRIEND_rec.columns[i]
    number_of_matches = len(get_big_match2(song_to_compare))
    
    if number_of_matches > 0:
        for j in range(number_of_matches):
            song_suggestion = get_big_match2(song_to_compare).index[j]
            cosine_similarity = FRIEND_rec[song_to_compare][song_suggestion]
            print(f'{song_suggestion} - {cosine_similarity}')
            if song_suggestion != song_to_compare:
                FRIEND_songs_for_playlist[song_suggestion] = [song_to_compare,
                                                             cosine_similarity]
#        display(pd.DataFrame(get_big_match2(song_to_compare)))
#         for j in range(number_of_matches):
#             song_row = FRIEND_rec[i]
#             print('(' + song_to_compare + ')')
#             display(pd.DataFrame(get_big_match2(song_to_compare)))
#             print()

Dancing In The Dark - Downtown Boys - 3Qi6WYMTatyrbqUWR5Ije9 - 0.9133559771616104
To You - Young Wonder - 3SjziwsYsQAXcxRbzNyCgO - 0.9315868705341079
It's Not Fair - Screaming Females - 3HlnH2wG0UhA9DofkbDILG - 0.9435336566913392
It's Not Fair - Screaming Females - 3HlnH2wG0UhA9DofkbDILG - 0.9370843574152192
On the Ropes - The Honeycutters - 0I1HiDFi0obu18Dh5Kd4J2 - 0.9422325114094909
Wishing Well - Screaming Females - 61ZocnIQ5gnNbcqD3fdNmj - 0.9126561358012625
On Life and Living - GILT - 1hnkH4tgCsxeibUvemRVpA - 0.9453339878760141
Flip-Flops at a Funeral - The Callous Daoboys - 4PJFITWxRo1nHZD0ESW17Z - 0.9083628591876454
The Absolute Barnstormer - The Callous Daoboys - 3R2DPVDLoDQCdYVKgGyDSa - 0.9539164813873514
Never Fight A Man With A Perm - IDLES - 7BmWDAlFk1DCL60I435oaE - 0.9031555995489342
Morbid Stuff - PUP - 0A31i51V3omHTJMDCJxDfh - 0.9246559533242306
Rivermouth - Rising Appalachia - 7hWIVUy7rGVrYLHv0PMbRZ - 0.9624293337558888
Pusher - alt-J - 2H6NbSsccvcT1j4jwAjDmf - 0.946941

In [84]:
FRIEND_songs_for_playlist

{'Dancing In The Dark - Downtown Boys - 3Qi6WYMTatyrbqUWR5Ije9': ['Bile Bittersweet - Bad Idols - 3dijvgdt9Z7WnxvIHUAUbF',
  0.9133559771616104],
 'To You - Young Wonder - 3SjziwsYsQAXcxRbzNyCgO': ['Like a Joke - Bad Idols - 6bbPSlmwFf5pOvuuq9z6uf',
  0.9315868705341079],
 "It's Not Fair - Screaming Females - 3HlnH2wG0UhA9DofkbDILG": ['Breakdown Days - Bad Idols - 0reenPmynLrPz5pRWgrFsa',
  0.9370843574152192],
 'On the Ropes - The Honeycutters - 0I1HiDFi0obu18Dh5Kd4J2': ['Sticks and Stones - Bad Idols - 4mpHviiBvLRglnRuomAJKL',
  0.9422325114094909],
 'Wishing Well - Screaming Females - 61ZocnIQ5gnNbcqD3fdNmj': ['Sticks and Stones - Bad Idols - 4mpHviiBvLRglnRuomAJKL',
  0.9126561358012625],
 'On Life and Living - GILT - 1hnkH4tgCsxeibUvemRVpA': ['Gummy Worms - Bad Idols - 6UlzxveYqDo4CdTTA88bSC',
  0.9453339878760141],
 'Never Fight A Man With A Perm - IDLES - 7BmWDAlFk1DCL60I435oaE': ['The Absolute Barnstormer - The Callous Daoboys - 3R2DPVDLoDQCdYVKgGyDSa',
  0.9031555995489342],
 

In [85]:
FRIEND_playlist = pd.DataFrame.from_dict(FRIEND_songs_for_playlist, orient='index', 
                                                  columns=['song matched on', 
                                                           'cosine similarity']).reset_index()

In [86]:
FRIEND_playlist = FRIEND_playlist.sort_values('cosine similarity', ascending=False).reset_index()

In [87]:
FRIEND_playlist.drop('level_0', axis=1, inplace=True)

In [88]:
FRIEND_playlist

,index,song matched on,cosine similarity
0,Rivermouth - Rising Appalachia - 7hWIVUy7rGVrY...,january - Insignificant Other - 2rFB2r05YeRpsW...,0.962429
1,Darlin Ukelele - Jolie Holland - 1vdjHLWHmUJCx...,What Happened (To the Dead) - Stagbriar - 4lMF...,0.951759
2,Like Eating Glass - Ladytron Zapatista Mix - B...,Scrooged - RVIVR - 6iUjRWLpMIjzzEWAwVbX1s,0.950942
3,Cough It Out - The Front Bottoms - 3K4y191l94w...,The Tide - RVIVR - 75P4CioGuCcFo0FSrBBsWn,0.946988
4,Pusher - alt-J - 2H6NbSsccvcT1j4jwAjDmf,january - Insignificant Other - 2rFB2r05YeRpsW...,0.946941
5,On Life and Living - GILT - 1hnkH4tgCsxeibUvem...,Gummy Worms - Bad Idols - 6UlzxveYqDo4CdTTA88bSC,0.945334
6,Fold - José González - 1h1nr0Nn5diE5S7PRUvjjc,Pause - RVIVR - 20TT9Ym6sF9gGDp6EGwziF,0.944061
7,My Father's Father - The Civil Wars - 7LJ0xFri...,january - Insignificant Other - 2rFB2r05YeRpsW...,0.942676
8,On the Ropes - The Honeycutters - 0I1HiDFi0obu...,Sticks and Stones - Bad Idols - 4mpHviiBvLRgln...,0.942233
9,New York - The Milk Carton Kids - 3VkSuzqfZJhr...,january - Insignificant Other - 2rFB2r05YeRpsW...,0.939824


In [89]:
playlist_length = FRIEND_playlist.shape[0]
playlist_length

55

In [91]:
playlist_ids = []

for i in range(playlist_length):
    suggested_track_title = FRIEND_playlist['index'][i].split('-')[0].strip()
    suggested_track_artist = FRIEND_playlist['index'][i].split('-')[1].strip()
    suggested_track_id = FRIEND_playlist['index'][i].split('-')[2].strip()
    playlist_ids.append(suggested_track_id)
    

In [92]:
playlist_ids

['7hWIVUy7rGVrYLHv0PMbRZ',
 '1vdjHLWHmUJCxaN7n1C4Xy',
 'Bloc Party',
 '3K4y191l94wf2t43qafbdV',
 'J',
 '1hnkH4tgCsxeibUvemRVpA',
 '1h1nr0Nn5diE5S7PRUvjjc',
 '7LJ0xFri8p1uKhRnBSwjfZ',
 '0I1HiDFi0obu18Dh5Kd4J2',
 '3VkSuzqfZJhrFUOjCweAKL',
 '3HlnH2wG0UhA9DofkbDILG',
 '182',
 '2PcLKxo6d9yO8rh04REWw6',
 '6en9s7fWhkJ9NaAh8MffkD',
 '3SjziwsYsQAXcxRbzNyCgO',
 '5OIE4Q3GWnyZSYFTbKEKDH',
 '2pDot0pyAGLxLjaznAUYnL',
 '0mLK2vXwuW8KDZ8XHvztkR',
 '5w5LFscgvImnXfXeGt6DME',
 '0mhFWJ1HkT8pTeqdlNRxvA',
 '0A31i51V3omHTJMDCJxDfh',
 '1drUAdX3NY4dTieuxdedLY',
 '5KsZSksgeYi3RxIoAAcy5l',
 '3uOxi3XD2dfSlUy3q1r73V',
 '3hzxPpZ5EIHWGEjfEaowSU',
 '4fBccEs6Ao7Dy8jJSrFjms',
 '49YQ9Mq8CKelQsmjdNiICu',
 '2UJ2JvrxU5Ue1y08o5TzqJ',
 'The Civil Wars',
 '3Qi6WYMTatyrbqUWR5Ije9',
 '3OOFEF20WqtsUPcRbPY3L7',
 '61ZocnIQ5gnNbcqD3fdNmj',
 '6EVyoECakDX7q7xIaMtNmM',
 'The Civil Wars',
 '5pBa5Jp4qqUlNeOnxJg7cD',
 '2JWSJruU0fLaNN3fqrLGwV',
 '5sIweazpg4JYBjyxABUx9y',
 '3kR1DF3AAFCzOtzMIErJ4H',
 '0MiqWcN3MF6FtkiMsIDWmI',
 '5ypWqYw34KK2B

# Create a Playlist???

In [ ]:
# user_playlist_create(user, name, public=True, description='')

In [ ]:
# username = 'MY_USERNAME'
# scope = 'user-read-private user-read-email user-library-read playlist-read-private playlist-modify-private playlist-modify-public'

# if len(sys.argv) > 1:
#     username = sys.argv[1]
# else:
#     print("Usage: %s username" % (sys.argv[0],))
#     sys.exit()

# token = util.prompt_for_user_token(username, 
#                                    scope,
#                                   client_id=SPOTIFY_CLIENT_ID,
#                                   client_secret=SPOTIFY_CLIENT_SECRET,
#                                   redirect_uri=SPOTIFY_REDIRECT_URI)

# all_saved_tracks = {}

# if token:
#     sp = spotipy.Spotify(auth=token)
#     sp.user_playlist_create(username, name='Recommended for FRIEND', public=False)

In [ ]:
full_df[(full_df['title'] == suggested_track_title) & ((full_df['artist'] == suggested_track_artist))]['track_id'].index

In [ ]:
full_df[(full_df['title'] == suggested_track_title) & ((full_df['artist'] == suggested_track_artist))]['track_id']

In [ ]:
get_big_match2('Stuff That I Like - Bomb the Music Industry!')

In [ ]:
FRIEND_rec['Stuff That I Like - Bomb the Music Industry!']['Homesick at Space Camp - Fall Out Boy']

In [ ]:
get_big_match2('Sally II - Hop Along').index[0]

In [ ]:
get_big_match2('Sally II - Hop Along').index[1]

In [ ]:
for i in range(number_of_tracks):
    song_to_compare = FRIEND_rec.columns[i]
    number_of_matches = len(get_big_match2(song_to_compare))
    print(f'{song_column}: {number_of_matches}')
    if number_of_matches > 0:
        for j in range(number_of_matches):
            song_row = FRIEND_rec[i]
            print('(' + song_to_compare + ')')
            display(pd.DataFrame(get_big_match2(song_to_compare)))
            print()

In [ ]:
for i in range(number_of_tracks):
    song_column = FRIEND_rec.columns[i]
    print(song_column)
#     if number_of_matches > 0:
#         for j in range(number_of_tracks):
#             song_row = get_big_match2('Sally II - Hop Along').index[0]
#             print('(' + song_to_compare + ')')
#             display(pd.DataFrame(get_big_match2(song_to_compare)))
#             print()

## What if we -include- matches between libraries?

In [93]:
def get_big_match3(song):
    return FRIEND_rec[FRIEND_rec[song] > .9][[song]].sort_values(by=song, ascending=False)


In [94]:
FRIEND_rec.shape

(3105, 167)

In [95]:
range(number_of_tracks)

range(0, 167)

In [96]:
# FRIEND_rec[0]

In [97]:
for i in range(167):
    song_to_compare = FRIEND_rec.columns[i]
    print('(' + song_to_compare + ')')
    print(len(get_big_match3(song_to_compare)))
    display(pd.DataFrame(get_big_match3(song_to_compare)))
    print()

(Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m)
0


track_id,Ex Cop - Bad Idols - 4YvvrIJy1yuLrH46guye4m
track_id,



(Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNgRqNvzuZ)
0


track_id,Freedom to Waste - Bad Idols - 2yAjekT6CrC6ZNgRqNvzuZ
track_id,



(Guilt - Bad Idols - 3t0P1KfxMrJBqSe4kCm8hU)
0


...

track_id,Bike Test 1 2 3 - Bomb the Music Industry! - 1AOZ7vnaMCy9yx59xiE04f
track_id,



(Bipolar Planes - This Is My Fist - 2zgaBa7mzGyww2lsIULIqY)
0


track_id,Bipolar Planes - This Is My Fist - 2zgaBa7mzGyww2lsIULIqY
track_id,



(Pray for Rain - The Ergs! - 6kpJGFFVsloSGd6PbEeve9)
0


track_id,Pray for Rain - The Ergs! - 6kpJGFFVsloSGd6PbEeve9
track_id,


In [105]:
FRIEND_songs_for_playlist = {}

for i in range(number_of_tracks):
    song_to_compare = FRIEND_rec.columns[i]
    number_of_matches = len(get_big_match3(song_to_compare))
    
    if number_of_matches > 0:
        for j in range(number_of_matches):
            song_suggestion = get_big_match3(song_to_compare).index[j]
            cosine_similarity = FRIEND_rec[song_to_compare][song_suggestion]
            FRIEND_songs_for_playlist[song_suggestion] = [song_to_compare,
                                                             cosine_similarity]
#        display(pd.DataFrame(get_big_match3(song_to_compare)))
#         for j in range(number_of_matches):
#             song_row = FRIEND_rec[i]
#             print('(' + song_to_compare + ')')
#             display(pd.DataFrame(get_big_match3(song_to_compare)))
#             print()

In [106]:
FRIEND_songs_for_playlist

{'Dancing In The Dark - Downtown Boys - 3Qi6WYMTatyrbqUWR5Ije9': ['Bile Bittersweet - Bad Idols - 3dijvgdt9Z7WnxvIHUAUbF',
  0.9133559771616104],
 'To You - Young Wonder - 3SjziwsYsQAXcxRbzNyCgO': ['Like a Joke - Bad Idols - 6bbPSlmwFf5pOvuuq9z6uf',
  0.9315868705341079],
 "It's Not Fair - Screaming Females - 3HlnH2wG0UhA9DofkbDILG": ['Breakdown Days - Bad Idols - 0reenPmynLrPz5pRWgrFsa',
  0.9370843574152192],
 'On the Ropes - The Honeycutters - 0I1HiDFi0obu18Dh5Kd4J2': ['Sticks and Stones - Bad Idols - 4mpHviiBvLRglnRuomAJKL',
  0.9422325114094909],
 'Wishing Well - Screaming Females - 61ZocnIQ5gnNbcqD3fdNmj': ['Sticks and Stones - Bad Idols - 4mpHviiBvLRglnRuomAJKL',
  0.9126561358012625],
 'On Life and Living - GILT - 1hnkH4tgCsxeibUvemRVpA': ['Gummy Worms - Bad Idols - 6UlzxveYqDo4CdTTA88bSC',
  0.9453339878760141],
 'Flip-Flops at a Funeral - The Callous Daoboys - 4PJFITWxRo1nHZD0ESW17Z': ['Flip-Flops at a Funeral - The Callous Daoboys - 4PJFITWxRo1nHZD0ESW17Z',
  0.9083628591876

In [107]:
FRIEND_playlist = pd.DataFrame.from_dict(FRIEND_songs_for_playlist, orient='index', 
                                                  columns=['song matched on', 
                                                           'cosine similarity']).reset_index()

In [108]:
FRIEND_playlist = FRIEND_playlist.sort_values('cosine similarity', ascending=False).reset_index()

In [109]:
FRIEND_playlist.drop('level_0', axis=1, inplace=True)

In [110]:
FRIEND_playlist

,index,song matched on,cosine similarity
0,Rivermouth - Rising Appalachia - 7hWIVUy7rGVrY...,january - Insignificant Other - 2rFB2r05YeRpsW...,0.962429
1,The Absolute Barnstormer - The Callous Daoboys...,The Absolute Barnstormer - The Callous Daoboys...,0.953916
2,Darlin Ukelele - Jolie Holland - 1vdjHLWHmUJCx...,What Happened (To the Dead) - Stagbriar - 4lMF...,0.951759
3,Like Eating Glass - Ladytron Zapatista Mix - B...,Scrooged - RVIVR - 6iUjRWLpMIjzzEWAwVbX1s,0.950942
4,Cough It Out - The Front Bottoms - 3K4y191l94w...,The Tide - RVIVR - 75P4CioGuCcFo0FSrBBsWn,0.946988
5,Pusher - alt-J - 2H6NbSsccvcT1j4jwAjDmf,january - Insignificant Other - 2rFB2r05YeRpsW...,0.946941
6,On Life and Living - GILT - 1hnkH4tgCsxeibUvem...,Gummy Worms - Bad Idols - 6UlzxveYqDo4CdTTA88bSC,0.945334
7,Fold - José González - 1h1nr0Nn5diE5S7PRUvjjc,Pause - RVIVR - 20TT9Ym6sF9gGDp6EGwziF,0.944061
8,january - Insignificant Other - 2rFB2r05YeRpsW...,january - Insignificant Other - 2rFB2r05YeRpsW...,0.942884
9,My Father's Father - The Civil Wars - 7LJ0xFri...,january - Insignificant Other - 2rFB2r05YeRpsW...,0.942676


In [111]:
playlist_length = FRIEND_playlist.shape[0]
playlist_length

58